In [2]:
# !pip install -r requirements.txt
# !pip install Pillow
# !pip install rembg
!pip install rembg[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 782.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 855.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached wheel-0.41.2-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 4.2 MB/s eta 0

In [1]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [2]:
df = pd.read_excel('chaussuretresbonetat.xlsx') 
df2 = pd.read_excel('chaussurebonetat.xlsx') 
# df3 = pd.read_excel('chaussuresatisfaisant.xlsx')
df4 = pd.read_excel('chaussureneufsansetiquette.xlsx')

In [3]:
df = df.iloc[:,4:6]
df2 = df2.iloc[:,4:6]
# df3 = df3.iloc[:,4:6]
df4 = df4.iloc[:,4:6]

In [4]:
# frames = [df,df2,df3,df4]
frames = [df,df2,df4]
result = pd.concat(frames,ignore_index=True)
result.to_csv('test.csv')

In [5]:
from rembg import remove
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed


imageLst = []
labelLst = []

def download_and_process(url, label):
    try:
        resp = session.get(url)
        img = Image.open(BytesIO(resp.content))
        
        # Removing background
        img_bytes = BytesIO(resp.content).read()
        img_no_bg_bytes = remove(img_bytes)
        img_no_bg = Image.open(BytesIO(img_no_bg_bytes))
        
        img_np = np.array(img_no_bg)
        img_resized = cv2.resize(img_np, (150, 150))
        return img_resized, label
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

with requests.Session() as session:
    with ThreadPoolExecutor(max_workers=20) as executor:
        future_to_url = {executor.submit(download_and_process, url, label): url for url, label in zip(result['image-src'], result['Etat'])}
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            img, label = future.result()
            if img is not None:
                imageLst.append(img)
                labelLst.append(label)

x = np.array(imageLst)
y = np.array(labelLst)


  2%|▏         | 194/9285 [01:26<1:07:17,  2.25it/s]


In [ ]:
# from concurrent.futures import ThreadPoolExecutor, as_completed


# imageLst = []
# labelLst = []

# def download_and_process(url, label):
#     try:
#         resp = session.get(url)
#         img = Image.open(BytesIO(resp.content))
#         img_np = np.array(img)
#         img_resized = cv2.resize(img_np, (150, 150))
#         return img_resized, label
#     except:
#         return None, None

# with requests.Session() as session:
#     with ThreadPoolExecutor() as executor:
#         future_to_url = {executor.submit(download_and_process, url, label): url for url, label in zip(result['image-src'], result['Etat'])}
#         for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
#         # for future in as_completed(future_to_url):
#             img, label = future.result()
#             if img is not None:
#                 imageLst.append(img)
#                 labelLst.append(label)

# x = np.array(imageLst)
# y = np.array(labelLst)

100%|██████████| 9285/9285 [01:59<00:00, 77.52it/s] 


In [ ]:
# imageLst = []
# labelLst = []

# with requests.Session() as session:
#     for i, url in enumerate(tqdm(result['image-src'])):
#         try:
#             resp = session.get(url)
#             img = Image.open(BytesIO(resp.content))
#             img_np = np.array(img)
#             img_resized = cv2.resize(img_np, (150, 150))
#             imageLst.append(img_resized)
#             labelLst.append(result['Etat'][i])
#         except:
#             pass

# # Convertir les listes en tableaux NumPy
# x = np.array(imageLst)
# y = np.array(labelLst)



  0%|          | 0/12381 [00:00<?, ?it/s]

 11%|█         | 1363/12381 [04:28<26:02,  7.05it/s]  

In [ ]:
print(len(x))
print(len(y))

9284
9284


In [ ]:
np.save("array_X.npy", x)
np.save("array_Y.npy", y)

In [ ]:
x = np.load("array_X.npy")
y = np.load("array_Y.npy")

In [ ]:
# Encodage des étiquettes
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded)

In [ ]:
print(label_encoder.classes_)

['bon etat' 'neuf sans etiquette' 'très bon etat']


In [ ]:
! pip install tensorflow-macos
! pip install tensorflow-metal

In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())


['/device:CPU:0', '/device:GPU:0']


2023-10-19 16:02:02.228992: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-10-19 16:02:02.229022: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-19 16:02:02.229032: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-19 16:02:02.229083: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-19 16:02:02.229109: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


# Diviser les données en ensembles d'entraînement, de validation et de test
x_train, x_temp, y_train, y_temp = train_test_split(x, y_one_hot, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Charger VGG16 pré-entraîné sur ImageNet et sans les couches entièrement connectées
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Geler les couches du modèle de base
for layer in base_model.layers:
    layer.trainable = False

# Créer le modèle personnalisé
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))  # Utiliser 'softmax' pour 3 ou 4 classes à modifier

# Compilation du modèle
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Entraînement
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Évaluer le modèle sur les données de test
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))


Epoch 1/10


2023-10-19 16:04:38.761127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2023-10-19 16:04:38.788479: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


204/204 [==============================] - 27s 121ms/step - loss: 53.7221 - accuracy: 0.3850 - val_loss: 24.2061 - val_accuracy: 0.4322
Epoch 2/10
204/204 [==============================] - 21s 104ms/step - loss: 26.0849 - accuracy: 0.4817 - val_loss: 21.4681 - val_accuracy: 0.4681
Epoch 3/10
204/204 [==============================] - 21s 104ms/step - loss: 22.3643 - accuracy: 0.5142 - val_loss: 33.7368 - val_accuracy: 0.3920
Epoch 4/10
204/204 [==============================] - 21s 104ms/step - loss: 18.6151 - accuracy: 0.5502 - val_loss: 15.2760 - val_accuracy: 0.5190
Epoch 5/10
204/204 [==============================] - 21s 104ms/step - loss: 16.8712 - accuracy: 0.5903 - val_loss: 29.1276 - val_accuracy: 0.4745
Epoch 6/10
204/204 [==============================] - 21s 104ms/step - loss: 15.8442 - accuracy: 0.6060 - val_loss: 19.1738 - val_accuracy: 0.5219
Epoch 7/10
204/204 [==============================] - 21s 104ms/step - loss: 14.3452 - accuracy: 0.6242 - val_loss: 40.9405 - val

In [18]:
# from tensorflow.keras.optimizers import Adam

# # Liste des hyperparamètres à tester
# learning_rates = [0.001, 0.01]
# batch_sizes = [16, 32]

# # Boucle pour tester chaque combinaison
# for lr in tqdm(learning_rates):
#     for batch_size in batch_sizes:
#         print(f"Entraînement avec learning rate = {lr} et batch size = {batch_size}")

#         # Créer le modèle personnalisé comme précédemment
#         model = models.Sequential()
#         model.add(base_model)
#         model.add(layers.Flatten())
#         model.add(layers.Dense(256, activation='relu'))
#         model.add(layers.Dropout(0.5))
#         model.add(layers.Dense(4, activation='softmax'))  # Utiliser 'softmax' pour 4 classes

#         # Compilation du modèle avec le taux d'apprentissage courant
#         model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

#         # Entraînement avec la taille du batch courant
#         history = model.fit(x_train, y_train, epochs=10, batch_size=batch_size, validation_data=(x_val, y_val))

#         # Évaluer le modèle sur les données de test
#         test_loss, test_accuracy = model.evaluate(x_test, y_test)
#         print(f"Test accuracy avec learning rate = {lr} et batch size = {batch_size}: {test_accuracy * 100:.2f}%")

In [16]:
# Sauvegarder le modèle entier
model.save("my_Fmodel.h5")


In [34]:
# image = cv2.imread("chauss.jpeg")
# print(image)

In [17]:
!ls -lh chauss.jpeg

-rw-r--r--@ 1 athena  staff   5.9K Oct 16 12:50 chauss.jpeg


In [14]:
import cv2
import numpy as np

# Charger une image avec OpenCV
# image = cv2.imread("chauss.jpeg")
# image = cv2.imread("chauNeuv.jpeg")
# image = cv2.imread("chaBof.jpeg")
image = cv2.imread("/Users/athena/Documents/ecole/vintedLikeComputingVision/test/1664263706.jpeg")

# Redimensionner l'image à 150x150
image_resized = cv2.resize(image, (150, 150))

# Normaliser l'image
image_normalized = image_resized / 255.0

# Redimensionner l'image pour ajouter une dimension de batch
image_batch = np.expand_dims(image_normalized, axis=0)


In [15]:
# Effectuer une prédiction
predictions = model.predict(image_batch)

# Obtenir l'indice de la classe prédite
predicted_class = np.argmax(predictions[0])

print("Classe prédite:", predicted_class)


1/1 [==============================] - 1s 760ms/step
Classe prédite: 0
